In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

data = pd.read_csv("data.csv")

data.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)

data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})

features = [
    'radius_mean', 'texture_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
    'symmetry_mean', 'radius_se', 'concave points_se', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'symmetry_worst', 'fractal_dimension_worst'
]

X = data[features]
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

layer_configs = [
    (64, 32),  
    (128, 64)  
]

batch_sizes = [32]  
epochs = 50
learning_rate = 0.001


results = []  #

class AccuracyLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        train_acc = logs.get('accuracy')
        print(f"Epoch {epoch + 1}: Train Accuracy = {train_acc:.4f}")

for layers in layer_configs:
    for batch_size in batch_sizes:
        print(f"\nTraining with Layers: {layers} and Batch Size: {batch_size}")
        
        model = Sequential([
            Input(shape=(X_train.shape[1],)),  
            Dense(layers[0], activation='relu'),  
            Dropout(0.3),
            Dense(layers[1], activation='relu'), 
            Dropout(0.3),
            Dense(1, activation='sigmoid')  
        ])

        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        history = model.fit(X_train, y_train,
                            epochs=epochs,
                            batch_size=batch_size,
                            callbacks=[AccuracyLogger()],
                            verbose=0)

        test_accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
        print(f"Test Accuracy: {test_accuracy:.4f}")

        results.append({
            'layers': layers,
            'batch_size': batch_size,
            'train_accuracy': history.history['accuracy'][-1],
            'test_accuracy': test_accuracy
        })

import pandas as pd

results_df = pd.DataFrame(results)
print("\nFinal Results:")
print(results_df)


Training with Layers: (64, 32) and Batch Size: 32
Epoch 1: Train Accuracy = 0.5890
Epoch 2: Train Accuracy = 0.8637
Epoch 3: Train Accuracy = 0.9055
Epoch 4: Train Accuracy = 0.9209
Epoch 5: Train Accuracy = 0.9363
Epoch 6: Train Accuracy = 0.9560
Epoch 7: Train Accuracy = 0.9626
Epoch 8: Train Accuracy = 0.9516
Epoch 9: Train Accuracy = 0.9604
Epoch 10: Train Accuracy = 0.9604
Epoch 11: Train Accuracy = 0.9538
Epoch 12: Train Accuracy = 0.9604
Epoch 13: Train Accuracy = 0.9516
Epoch 14: Train Accuracy = 0.9582
Epoch 15: Train Accuracy = 0.9648
Epoch 16: Train Accuracy = 0.9626
Epoch 17: Train Accuracy = 0.9604
Epoch 18: Train Accuracy = 0.9604
Epoch 19: Train Accuracy = 0.9626
Epoch 20: Train Accuracy = 0.9560
Epoch 21: Train Accuracy = 0.9670
Epoch 22: Train Accuracy = 0.9670
Epoch 23: Train Accuracy = 0.9670
Epoch 24: Train Accuracy = 0.9670
Epoch 25: Train Accuracy = 0.9648
Epoch 26: Train Accuracy = 0.9714
Epoch 27: Train Accuracy = 0.9692
Epoch 28: Train Accuracy = 0.9758
Epoch 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from tqdm import tqdm

data = pd.read_csv("data.csv")
data.drop(['id', 'Unnamed: 32'], axis=1, inplace=True) 
data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0}) 

features = [
    'radius_mean', 'texture_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
    'symmetry_mean', 'radius_se', 'concave points_se', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'symmetry_worst', 'fractal_dimension_worst'
]

X = data[features]
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_model(learning_rate, dropout_rate):
    model = Sequential([
        Input(shape=(X_train.shape[1],)), 
        Dense(64, activation='relu'),    
        Dropout(dropout_rate),            
        Dense(32, activation='relu'),    
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')   
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


model = KerasClassifier(
    model=create_model,
    model__learning_rate=0.001, 
    model__dropout_rate=0.3,   
    verbose=0
)


param_grid = {
    'model__learning_rate': [0.1],  
    'model__dropout_rate': [0.2],        
    'batch_size': [8],                       
    'epochs': [30]                                
}


from itertools import product

param_combinations = list(product(
    param_grid['model__learning_rate'],
    param_grid['model__dropout_rate'],
    param_grid['batch_size'],
    param_grid['epochs']
))

print(f"Total parameter combinations to test: {len(param_combinations)}")


results = []

for i, (lr, dr, bs, ep) in enumerate(tqdm(param_combinations, desc="Hyperparameter Tuning Progress"), start=1):
    temp_grid = {
        'model__learning_rate': [lr],
        'model__dropout_rate': [dr],
        'batch_size': [bs],
        'epochs': [ep]
    }

    grid = GridSearchCV(
        estimator=model,
        param_grid=temp_grid,
        cv=3,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
    
    grid_result = grid.fit(X_train, y_train)

    results.append({
        'learning_rate': lr,
        'dropout_rate': dr,
        'batch_size': bs,
        'epochs': ep,
        'best_score': grid_result.best_score_
    })

results_df = pd.DataFrame(results)
best_row = results_df.loc[results_df['best_score'].idxmax()]

print("\nBest Hyperparameters and Score:")
print(best_row)

best_model = KerasClassifier(
    model=create_model,
    model__learning_rate=best_row['learning_rate'],
    model__dropout_rate=best_row['dropout_rate'],
    verbose=0
)

best_model.fit(X_train, y_train, batch_size=int(best_row['batch_size']), epochs=int(best_row['epochs']), verbose=0)
test_accuracy = best_model.score(X_test, y_test)
print(f"Test Accuracy with Best Hyperparameters: {test_accuracy:.4f}")


Total parameter combinations to test: 1


Hyperparameter Tuning Progress: 100%|█████████████| 1/1 [00:44<00:00, 44.77s/it]



Best Hyperparameters and Score:
learning_rate     0.10000
dropout_rate      0.20000
batch_size        8.00000
epochs           30.00000
best_score        0.96041
Name: 0, dtype: float64
Test Accuracy with Best Hyperparameters: 0.9386


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from tqdm import tqdm

data = pd.read_csv("data.csv")
data.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)  
data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0}) 

features = [
    'radius_mean', 'texture_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
    'symmetry_mean', 'radius_se', 'concave points_se', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'symmetry_worst', 'fractal_dimension_worst'
]

X = data[features]
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_model(learning_rate, dropout_rate):
    model = Sequential([
        Input(shape=(X_train.shape[1],)), 
        Dense(64, activation='relu'),   
        Dropout(dropout_rate),           
        Dense(32, activation='relu'),    
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')   
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = KerasClassifier(
    model=create_model,
    model__learning_rate=0.001, 
    model__dropout_rate=0.3,   
    verbose=0
)

param_grid = {
    'model__learning_rate': [1.0, 0.1, 0.001, 0.01, 0.0001],  
    'model__dropout_rate': [0.2],        
    'batch_size': [8],                       
    'epochs': [30]                                
}


from itertools import product

param_combinations = list(product(
    param_grid['model__learning_rate'],
    param_grid['model__dropout_rate'],
    param_grid['batch_size'],
    param_grid['epochs']
))

print(f"Total parameter combinations to test: {len(param_combinations)}")


results = []

for i, (lr, dr, bs, ep) in enumerate(tqdm(param_combinations, desc="Hyperparameter Tuning Progress"), start=1):
    temp_grid = {
        'model__learning_rate': [lr],
        'model__dropout_rate': [dr],
        'batch_size': [bs],
        'epochs': [ep]
    }


    grid = GridSearchCV(
        estimator=model,
        param_grid=temp_grid,
        cv=3,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
    
    grid_result = grid.fit(X_train, y_train)

    results.append({
        'learning_rate': lr,
        'dropout_rate': dr,
        'batch_size': bs,
        'epochs': ep,
        'best_score': grid_result.best_score_
    })

results_df = pd.DataFrame(results)
best_row = results_df.loc[results_df['best_score'].idxmax()]

print("\nBest Hyperparameters and Score:")
print(best_row)


best_model = KerasClassifier(
    model=create_model,
    model__learning_rate=best_row['learning_rate'],
    model__dropout_rate=best_row['dropout_rate'],
    verbose=0
)

best_model.fit(X_train, y_train, batch_size=int(best_row['batch_size']), epochs=int(best_row['epochs']), verbose=0)
test_accuracy = best_model.score(X_test, y_test)
print(f"Test Accuracy with Best Hyperparameters: {test_accuracy:.4f}")


Total parameter combinations to test: 5


Hyperparameter Tuning Progress: 100%|█████████████| 5/5 [04:38<00:00, 55.64s/it]



Best Hyperparameters and Score:
learning_rate     0.001000
dropout_rate      0.200000
batch_size        8.000000
epochs           30.000000
best_score        0.975834
Name: 2, dtype: float64
Test Accuracy with Best Hyperparameters: 0.9561


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from tqdm import tqdm

data = pd.read_csv("data.csv")
data.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)  
data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0}) 

features = [
    'radius_mean', 'texture_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
    'symmetry_mean', 'radius_se', 'concave points_se', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'symmetry_worst', 'fractal_dimension_worst'
]

X = data[features]
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_model(learning_rate, dropout_rate):
    model = Sequential([
        Input(shape=(X_train.shape[1],)), 
        Dense(64, activation='relu'),    
        Dropout(dropout_rate),           
        Dense(32, activation='relu'),     
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')    
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = KerasClassifier(
    model=create_model,
    model__learning_rate=0.001,  
    model__dropout_rate=0.3,    
    verbose=0
)

param_grid = {
    'model__learning_rate': [0.1],  
    'model__dropout_rate': [0.2, 0.3, 0.4],        
    'batch_size': [8],                       
    'epochs': [30]                                
}


from itertools import product

param_combinations = list(product(
    param_grid['model__learning_rate'],
    param_grid['model__dropout_rate'],
    param_grid['batch_size'],
    param_grid['epochs']
))

print(f"Total parameter combinations to test: {len(param_combinations)}")


results = []

for i, (lr, dr, bs, ep) in enumerate(tqdm(param_combinations, desc="Hyperparameter Tuning Progress"), start=1):
    temp_grid = {
        'model__learning_rate': [lr],
        'model__dropout_rate': [dr],
        'batch_size': [bs],
        'epochs': [ep]
    }


    grid = GridSearchCV(
        estimator=model,
        param_grid=temp_grid,
        cv=3,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
    
    grid_result = grid.fit(X_train, y_train)

    results.append({
        'learning_rate': lr,
        'dropout_rate': dr,
        'batch_size': bs,
        'epochs': ep,
        'best_score': grid_result.best_score_
    })


results_df = pd.DataFrame(results)
best_row = results_df.loc[results_df['best_score'].idxmax()]

print("\nBest Hyperparameters and Score:")
print(best_row)


best_model = KerasClassifier(
    model=create_model,
    model__learning_rate=best_row['learning_rate'],
    model__dropout_rate=best_row['dropout_rate'],
    verbose=0
)


best_model.fit(X_train, y_train, batch_size=int(best_row['batch_size']), epochs=int(best_row['epochs']), verbose=0)
test_accuracy = best_model.score(X_test, y_test)
print(f"Test Accuracy with Best Hyperparameters: {test_accuracy:.4f}")


Total parameter combinations to test: 3


Hyperparameter Tuning Progress: 100%|█████████████| 3/3 [02:16<00:00, 45.50s/it]



Best Hyperparameters and Score:
learning_rate     0.100000
dropout_rate      0.300000
batch_size        8.000000
epochs           30.000000
best_score        0.964811
Name: 1, dtype: float64
Test Accuracy with Best Hyperparameters: 0.9649


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from tqdm import tqdm

data = pd.read_csv("data.csv")
data.drop(['id', 'Unnamed: 32'], axis=1, inplace=True) 
data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0}) 

features = [
    'radius_mean', 'texture_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
    'symmetry_mean', 'radius_se', 'concave points_se', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'symmetry_worst', 'fractal_dimension_worst'
]

X = data[features]
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_model(learning_rate, dropout_rate):
    model = Sequential([
        Input(shape=(X_train.shape[1],)), 
        Dense(64, activation='relu'),    
        Dropout(dropout_rate),          
        Dense(32, activation='relu'),    
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')   
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = KerasClassifier(
    model=create_model,
    model__learning_rate=0.001, 
    model__dropout_rate=0.3,    
    verbose=0
)


param_grid = {
    'model__learning_rate': [0.1],  
    'model__dropout_rate': [0.2],        
    'batch_size': [8, 16, 32],                       
    'epochs': [30]                                
}


from itertools import product

param_combinations = list(product(
    param_grid['model__learning_rate'],
    param_grid['model__dropout_rate'],
    param_grid['batch_size'],
    param_grid['epochs']
))

print(f"Total parameter combinations to test: {len(param_combinations)}")


results = []

for i, (lr, dr, bs, ep) in enumerate(tqdm(param_combinations, desc="Hyperparameter Tuning Progress"), start=1):
    temp_grid = {
        'model__learning_rate': [lr],
        'model__dropout_rate': [dr],
        'batch_size': [bs],
        'epochs': [ep]
    }

    grid = GridSearchCV(
        estimator=model,
        param_grid=temp_grid,
        cv=3,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
    
    grid_result = grid.fit(X_train, y_train)

    results.append({
        'learning_rate': lr,
        'dropout_rate': dr,
        'batch_size': bs,
        'epochs': ep,
        'best_score': grid_result.best_score_
    })

results_df = pd.DataFrame(results)
best_row = results_df.loc[results_df['best_score'].idxmax()]

print("\nBest Hyperparameters and Score:")
print(best_row)


best_model = KerasClassifier(
    model=create_model,
    model__learning_rate=best_row['learning_rate'],
    model__dropout_rate=best_row['dropout_rate'],
    verbose=0
)


best_model.fit(X_train, y_train, batch_size=int(best_row['batch_size']), epochs=int(best_row['epochs']), verbose=0)
test_accuracy = best_model.score(X_test, y_test)
print(f"Test Accuracy with Best Hyperparameters: {test_accuracy:.4f}")


Total parameter combinations to test: 3


Hyperparameter Tuning Progress: 100%|█████████████| 3/3 [01:30<00:00, 30.07s/it]



Best Hyperparameters and Score:
learning_rate     0.10000
dropout_rate      0.20000
batch_size        8.00000
epochs           30.00000
best_score        0.96041
Name: 0, dtype: float64
Test Accuracy with Best Hyperparameters: 0.9474


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from tqdm import tqdm

data = pd.read_csv("data.csv")
data.drop(['id', 'Unnamed: 32'], axis=1, inplace=True) 
data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0}) 

features = [
    'radius_mean', 'texture_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
    'symmetry_mean', 'radius_se', 'concave points_se', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'symmetry_worst', 'fractal_dimension_worst'
]

X = data[features]
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_model(learning_rate, dropout_rate):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),    
        Dropout(dropout_rate),           
        Dense(32, activation='relu'),     
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')   
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = KerasClassifier(
    model=create_model,
    model__learning_rate=0.001,  
    model__dropout_rate=0.3,   
    verbose=0
)


param_grid = {
    'model__learning_rate': [0.1],  
    'model__dropout_rate': [0.2],        
    'batch_size': [8],                       
    'epochs': [20, 30, 50, 70, 100, 200]                                
}


from itertools import product

param_combinations = list(product(
    param_grid['model__learning_rate'],
    param_grid['model__dropout_rate'],
    param_grid['batch_size'],
    param_grid['epochs']
))

print(f"Total parameter combinations to test: {len(param_combinations)}")

results = []

for i, (lr, dr, bs, ep) in enumerate(tqdm(param_combinations, desc="Hyperparameter Tuning Progress"), start=1):
    temp_grid = {
        'model__learning_rate': [lr],
        'model__dropout_rate': [dr],
        'batch_size': [bs],
        'epochs': [ep]
    }

    grid = GridSearchCV(
        estimator=model,
        param_grid=temp_grid,
        cv=3,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
    
    grid_result = grid.fit(X_train, y_train)

    results.append({
        'learning_rate': lr,
        'dropout_rate': dr,
        'batch_size': bs,
        'epochs': ep,
        'best_score': grid_result.best_score_
    })

results_df = pd.DataFrame(results)
best_row = results_df.loc[results_df['best_score'].idxmax()]

print("\nBest Hyperparameters and Score:")
print(best_row)

best_model = KerasClassifier(
    model=create_model,
    model__learning_rate=best_row['learning_rate'],
    model__dropout_rate=best_row['dropout_rate'],
    verbose=0
)

best_model.fit(X_train, y_train, batch_size=int(best_row['batch_size']), epochs=int(best_row['epochs']), verbose=0)
test_accuracy = best_model.score(X_test, y_test)
print(f"Test Accuracy with Best Hyperparameters: {test_accuracy:.4f}")


Total parameter combinations to test: 6


Hyperparameter Tuning Progress: 100%|████████████| 6/6 [10:56<00:00, 109.42s/it]



Best Hyperparameters and Score:
learning_rate     0.100000
dropout_rate      0.200000
batch_size        8.000000
epochs           30.000000
best_score        0.969226
Name: 1, dtype: float64
Test Accuracy with Best Hyperparameters: 0.9298
